In [9]:
import os

def get_next_directory(base_path="output/predict"):
    # Check if the base directory exists
    if not os.path.exists(base_path):
        return base_path
    else:
        # Find the next available numbered directory
        i = 2
        while os.path.exists(f"{base_path}{i}"):
            i += 1
        return f"{base_path}{i}"

In [15]:
HOME_DIR = os.getcwd()

# Create the output directory, if needed
parent_dir = os.path.dirname(HOME_DIR)
OUTPUT_DIR = get_next_directory(os.path.join(parent_dir, "output/predict"))

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Perform object detection

Perform the inference with the loaded model.

In [12]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load an official model

# Predict with the model
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

100%|██████████| 134k/134k [00:00<00:00, 2.20MB/s]

image 1/1 /Users/andreas/Documents/Projekte/Objekterkennung.yolo11/notebooks/bus.jpg: 640x480 4 persons, 1 bus, 49.5ms
Speed: 3.3ms preprocess, 49.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)


# Save cropped images

Saves images for each bounding box.

In [16]:
for result in results:
    result.save_crop(OUTPUT_DIR, "image")